In [ ]:
# full_valuation_from_discountingcashflows.py
%pip install rapidfuzz --quiet
 
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import yfinance as yf
from scipy.optimize import curve_fit
import warnings
from rapidfuzz import process, fuzz
import os
from datetime import datetime
from scipy import optimize
from scipy.stats import linregress
 
 
# Suppress warnings that are not critical for the valuation output
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=optimize.OptimizeWarning)
 
 
# ==============================
# CONFIGURABLE VARIABLES
# ==============================
TICKER = "G.MI"        # Stock ticker to analyze (e.g. "DIA.MI", "ENEL.MI", "AAPL")
YEARS = 10               # Number of projection years for DCF/EVA
TERMINAL_GROWTH = 0.02   # Long-term growth rate (perpetual growth)
RISK_FREE = 0.04         # Risk-free rate
MARKET_PREMIUM = 0.05   # Market risk premium
TAX_RATE = 0.21          # Tax rate to apply to the calculated WACC
MANUAL_EV_SALES = None  # Manual EV/Sales multiple (float), set to None to use industry average from local file
USE_FCF_AVG = None       # Set to True to use average FCF of last n years as base, otherwise uses latest year
FCF_AVG_YEARS = 3        # Number of years for average FCF calculation (if USE_FCF_AVG is True)
DATA_SCALE_FACTOR = 1_000_000 # Scaling factor for data from discountingcashflows.com ("In Millions")
BASE_URL = https://discountingcashflows.com/company/{ticker}/{statement}/
 
# Local file path for EV/Sales data
LOCAL_EV_SALES_FILE = "/content/drive/MyDrive/EV-SALES/EV_Sales_{last_year}.xlsx"
 
# Variables needed for regression
REGRESSION_YEARS = 15 # Number of historical fiscal years for regression
PROJECTION_YEARS = 10
 
# ==============================
 
 
# ------------------------------
# UTILITIES: Download HTML Table
# ------------------------------
def get_table_discounting(ticker, statement):
    """Downloads the first relevant table from discountingcashflows.com for a given statement."""
    url = BASE_URL.format(ticker=ticker, statement=statement)
    headers = {"User-Agent": "Mozilla/5.0"}
    r = requests.get(url, headers=headers, timeout=20)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "html.parser")
 
    tables = soup.find_all("table")
    if not tables:
        raise ValueError(f"No table found for {statement} on {url}")
    for t in tables:
        try:
            df = pd.read_html(str(t))[0]
            if df.shape[1] >= 2:
                return df
        except Exception:
            continue
    raise ValueError(f"No readable table found for {statement} on {url}")
 
 
def extract_series_from_row(df, keywords):
    """
    Searches for a row containing any of the keywords (list) and returns the list of values
    (from most recent to oldest as float numbers). Removes '-' and ','.
    Returns a list of values and a list of periods (column names).
    """
    periods = df.columns.tolist()[1:] # Assuming periods are column names after the first label column
    for kw in keywords:
        mask = df.iloc[:, 0].astype(str).str.contains(kw, case=False, na=False)
        if mask.any():
            row = df.loc[mask, :].iloc[0]
            vals = row.iloc[1:].astype(str).replace('-', np.nan).replace('', np.nan)
            cleaned_vals = []
            cleaned_periods = []
            for i, v in enumerate(vals):
                if pd.isna(v):
                    continue
                s = str(v).replace(",", "").replace("(", "-").replace(")", "").strip()
                try:
                    cleaned_vals.append(float(s))
                    cleaned_periods.append(periods[i])
                except:
                    continue
            if cleaned_vals:
                return cleaned_vals, cleaned_periods
    return None, None
 
 
# ------------------------------
# HELPER: Perform Regression and Projection (handles both Linear and Logarithmic)
# ------------------------------
def predict_regression(historical_data_with_ttm, historical_periods_with_ttm, projection_years, data_name, regression_type='linear'):
    """Performs specified regression on historical data (including TTM) and predicts future values."""
 
    # Create DataFrame, extract year, filter for fiscal years and TTM
    reg_df = pd.DataFrame({data_name: historical_data_with_ttm})
    reg_df['Period'] = historical_periods_with_ttm[:len(historical_data_with_ttm)]
 
    # Identify TTM row
    ttm_mask = reg_df['Period'].astype(str).str.contains('TTM', case=False, na=False)
    ttm_data = reg_df[ttm_mask].copy()
 
    # Filter for fiscal years (contain '20')
    fiscal_years_df = reg_df[reg_df['Period'].astype(str).str.contains('20', na=False)].copy()
 
    # Extract numeric year from fiscal years
    fiscal_years_df['Year_Numeric'] = fiscal_years_df['Period'].str.extract(r'(\d{4})').astype(float)
 
    # Sort fiscal years by year ascending
    fiscal_years_df = fiscal_years_df.sort_values(by='Year_Numeric').reset_index(drop=True)
 
    # Filter for the last REGRESSION_YEARS fiscal years
    if len(fiscal_years_df) > REGRESSION_YEARS:
        fiscal_years_df = fiscal_years_df.tail(REGRESSION_YEARS).reset_index(drop=True)
        print(f"Using last {REGRESSION_YEARS} fiscal years for {data_name} regression: {fiscal_years_df['Period'].tolist()}")
    elif len(fiscal_years_df) < 2 and ttm_data.empty:
         print(f"Not enough historical fiscal year data ({len(fiscal_years_df)}) for {data_name} regression (need at least 2 fiscal years or 1 fiscal year + TTM).")
         return None # Cannot perform regression
 
    # Combine selected fiscal years and TTM data for regression input
    regression_input_df = fiscal_years_df.copy()
    if not ttm_data.empty:
        # For regression, treat TTM as year 0 relative to the last fiscal year
        if not fiscal_years_df.empty:
            last_fiscal_year_numeric = regression_input_df['Year_Numeric'].max()
            ttm_year_numeric = last_fiscal_year_numeric + 1
        else:
            print(f"Not enough data for {data_name} regression (only TTM available).")
            return None
 
        ttm_data['Year_Numeric'] = ttm_year_numeric
        regression_input_df = pd.concat([regression_input_df, ttm_data], ignore_index=True)
 
 
    if len(regression_input_df) < 2:
         print(f"Not enough data points ({len(regression_input_df)}) for {data_name} regression (need at least 2 points).")
         return None
 
    # Use years since the first year of the regression data as the x-axis
    first_year_numeric = regression_input_df['Year_Numeric'].min()
    regression_input_df['Years_Since_Start'] = regression_input_df['Year_Numeric'] - first_year_numeric
 
 
    x = regression_input_df['Years_Since_Start'].values
    y = regression_input_df[data_name].values
 
    if regression_type == 'linear':
        # Perform Linear Regression using scipy.stats.linregress
        try:
            slope, intercept, r_value, p_value, std_err = linregress(x, y)
            print(f"\n{data_name} Linear Regression: {data_name} = {intercept:.2f} + {slope:.2f} * (Year - {first_year_numeric})")
 
            # Predict values for projection_years subsequent years
            last_input_year_numeric = regression_input_df['Year_Numeric'].max()
            projected_values = {}
            for i in range(1, projection_years + 1):
                projection_year = last_input_year_numeric + i
                years_since_start_proj = projection_year - first_year_numeric
                predicted_val = intercept + slope * years_since_start_proj
                projected_values[int(projection_year)] = predicted_val
 
            projected_series = pd.Series(projected_values)
            print(f"\nPredicted {data_name} for the next {projection_years} years (Linear Regression):")
            print(projected_series.round(2))
 
            return projected_series
 
        except Exception as e:
            print(f"Error during {data_name} Linear Regression: {e}")
            return None
 
    elif regression_type == 'logarithmic':
        # For logarithmic regression, exclude data points with data_name <= 0
        reg_df_positive = regression_input_df[regression_input_df[data_name] > 0].copy()
 
        if len(reg_df_positive) < 2:
            print(f"Not enough positive data points ({len(reg_df_positive)}) for {data_name} logarithmic regression (need at least 2 positive values).")
            return None # Cannot perform regression
 
        # Use years since the first year of the positive data as the x-axis for log regression
        first_positive_year_numeric = reg_df_positive['Year_Numeric'].min()
        # Calculate offset to make x-axis values positive for log regression if needed
        # Ensure the minimum x value is at least 1
        year_offset_log = 0
        if (reg_df_positive['Year_Numeric'] - first_positive_year_numeric).min() <= 0:
             year_offset_log = 1 - (reg_df_positive['Year_Numeric'] - first_positive_year_numeric).min()
 
        reg_df_positive['Years_Since_Start_Log'] = reg_df_positive['Year_Numeric'] - first_positive_year_numeric + year_offset_log
 
        x_log = reg_df_positive['Years_Since_Start_Log'].values
        y_log = reg_df_positive[data_name].values
 
        # Define the logarithmic function
        def log_func(x, a, b):
             return a + b * np.log(x)
 
        try:
            # Initial guess for parameters (a, b)
            initial_guess = [y_log[0], 0.1] if len(y_log) > 0 else [1.0, 0.1]
 
            params, covariance = curve_fit(log_func, x_log, y_log, p0=initial_guess)
            a_log, b_log = params
            print(f"\n{data_name} Logarithmic Regression: {data_name} = {a_log:.2f} + {b_log:.2f} * log(Year - {first_positive_year_numeric} + {year_offset_log})")
 
            # Predict values for projection_years subsequent years
            last_input_year_numeric = regression_input_df['Year_Numeric'].max()
            projected_values = {}
            for i in range(1, projection_years + 1):
                projection_year = last_input_year_numeric + i
                years_since_start_proj_log = projection_year - first_positive_year_numeric + year_offset_log
                if years_since_start_proj_log > 0:
                    predicted_val = log_func(years_since_start_proj_log, a_log, b_log)
                    projected_values[int(projection_year)] = predicted_val
                else:
                     projected_values[int(projection_year)] = np.nan
 
            projected_series = pd.Series(projected_values)
            print(f"\nPredicted {data_name} for the next {projection_years} years (Logarithmic Regression):")
            print(projected_series.round(2))
 
            return projected_series
 
        except RuntimeError as e:
            print(f"Error during {data_name} Logarithmic Regression curve_fit: {e}. This can happen if the data doesn't fit a log curve well.")
            return None
        except Exception as e:
            print(f"Error during {data_name} Logarithmic Regression: {e}")
            return None
    else:
        print(f"Error: Unknown regression type '{regression_type}'. Please use 'linear' or 'logarithmic'.")
        return None
 
 
# ------------------------------
# Load EV/Sales from local file
# ------------------------------
def load_ev_sales_from_local_file(file_path):
    """Loads EV/Sales data from a local Excel file."""
    try:
        df = pd.read_excel(file_path, engine='openpyxl')
        if 'Industry' not in df.columns:
            raise ValueError("Column 'Industry' not found in the Excel file.")
        if 'mean_EV_Sales' not in df.columns:
             raise ValueError("Column 'mean_EV_Sales' not found in the Excel file.")
        df.set_index('Industry', inplace=True)
        df.index.name = 'Industry'
        return df[['mean_EV_Sales']]
    except FileNotFoundError:
        raise FileNotFoundError(f"EV/Sales file not found at: {file_path}")
    except Exception as e:
        raise RuntimeError(f"Error loading EV/Sales data from local file ({file_path}): {e}")
 
 
def find_ev_sales_by_industry(industry, local_ev_sales_df, manual_ev_sales=None):
    """
    Searches for industry in the local EV/Sales table (fuzzy search) and returns mean EV/Sales (float).
    If not found and manual_ev_sales is provided, returns manual_ev_sales.
    """
    if industry is None or industry == "":
        if manual_ev_sales is not None:
            print("\nIndustry not available from Yahoo Finance. Using manual EV/Sales value:", manual_ev_sales)
            return manual_ev_sales
        raise ValueError("Industry not available from Yahoo Finance and manual_ev_sales not provided.")
 
    if local_ev_sales_df.index.name != 'Industry':
         if local_ev_sales_df.index.name is None or str(local_ev_sales_df.index.name).lower() != 'industry':
             raise RuntimeError("Local EV/Sales DataFrame must have 'Industry' as its index name.")
 
    industries_in_local_file = local_ev_sales_df.index.dropna().tolist()
 
    if not industries_in_local_file:
         raise ValueError("No industry names found in local EV/Sales file.")
 
    score_cutoff = 60
    match_result = process.extractOne(
        industry, industries_in_local_file, scorer=fuzz.ratio, score_cutoff=score_cutoff
    )
 
    if isinstance(match_result, tuple) and len(match_result) >= 2:
        match = match_result[0]
        score = match_result[1]
 
        try:
            val = local_ev_sales_df.loc[match, 'mean_EV_Sales']
            print(f"\nFuzzy match found for industry '{industry}': '{match}' (score: {score:.2f}). Using mean EV/Sales: {val}")
            return val
        except KeyError:
             raise RuntimeError(f"Internal error: Matched industry '{match}' not found in local EV/Sales DataFrame index.")
        except Exception as e:
             raise RuntimeError(f"Error accessing mean_EV_Sales value for matched industry '{match}': {e}")
    else:
        best_match_info = process.extractOne(industry, industries_in_local_file, scorer=fuzz.ratio)
        best_score = best_match_info[1] if isinstance(best_match_info, tuple) and len(best_match_info) > 1 else 0
 
        if manual_ev_sales is not None:
            print(f"\nNo good fuzzy match found for industry '{industry}' (best score: {best_score:.2f} below cutoff {score_cutoff}). Using manual EV/Sales value: {manual_ev_sales}")
            return manual_ev_sales
        else:
            raise ValueError(f"No good fuzzy match found for industry '{industry}' (best score: {best_score:.2f} below cutoff {score_cutoff}) and manual_ev_sales not provided.")
 
 
# ------------------------------
# WACC (from discountingcashflows statements + Yahoo fallback)
# ------------------------------
def calculate_wacc_discounting(ticker, risk_free=RISK_FREE, market_premium=MARKET_PREMIUM, tax_rate=TAX_RATE):
    try:
        bs = get_table_discounting(ticker, "balance-sheet-statement")
        is_df = get_table_discounting(ticker, "income-statement")
    except Exception as e:
        raise RuntimeError(f"Error downloading statements for WACC: {e}")
 
    bs = bs.set_index(bs.columns[0])
    is_df = is_df.set_index(is_df.columns[0])
 
    debt_keys = ["Total Debt", "Total debt", "Total liabilities", "Total Liabilities", "Total Debt & Leases", "Total Long Term Debt"]
    total_debt = 0.0
    for k in debt_keys:
        if k in bs.index:
            try:
                total_debt = float(str(bs.loc[k].dropna().values[0]).replace(",", "").replace("(", "-").replace(")", "")) * DATA_SCALE_FACTOR
                break
            except:
                continue
 
    cash_keys = ["Cash and Short Term Investments", "Cash & Equivalents", "Cash and cash equivalents", "Cash"]
    cash = 0.0
    for k in cash_keys:
        if k in bs.index:
            try:
                cash = float(str(bs.loc[k].dropna().values[0]).replace(",", "").replace("(", "-").replace(")", "")) * DATA_SCALE_FACTOR
                break
            except:
                continue
 
    interest_keys = ["Interest Expense", "interest expense", "Interest paid", "Interest Paid", "Net Non-Operating Interest"]
    interest_expense = None
    for k in interest_keys:
        if k in is_df.index:
            try:
                interest_expense = float(str(is_df.loc[k].dropna().values[0]).replace(",", "").replace("(", "-").replace(")", "")) * DATA_SCALE_FACTOR
                break
            except:
                continue
 
    stock = yf.Ticker(ticker)
    info = {}
    try:
        info = stock.info
    except Exception:
        info = {}
    equity_value = info.get("marketCap", None)
    shares_outstanding = info.get("sharesOutstanding", None)
    beta = info.get("beta", 1.0)
    current_price = info.get("regularMarketPrice", None)
 
    if shares_outstanding is None and equity_value and current_price and current_price > 0:
        try:
            shares_outstanding = int(round(equity_value / current_price))
        except:
            shares_outstanding = None
 
    if equity_value is None and shares_outstanding and current_price:
        equity_value = shares_outstanding * current_price
 
    cost_of_equity = risk_free + beta * market_premium
 
    cost_of_debt = 0.04
    if interest_expense is not None and total_debt > 0:
        cost_of_debt = abs(interest_expense) / total_debt
 
    ev = None
    if equity_value is not None:
        ev = equity_value + total_debt - cash
    else:
        book_equity = None
        book_keys = ["Total Equity", "Total shareholders' equity", "Total stockholders' equity", "Total Equity Attributable To Parent"]
        for k in bs.index:
            if k in bs.index:
                try:
                    book_equity = float(str(bs.loc[k].dropna().values[0]).replace(",", "").replace("(", "-").replace(")", "")) * DATA_SCALE_FACTOR
                    break
                except:
                    continue
        if book_equity is not None:
            equity_value = book_equity
            ev = equity_value + total_debt - cash
 
 
    if ev is None or ev == 0:
        raise RuntimeError("Unable to calculate EV (missing marketCap and book equity).")
 
    e_weight = equity_value / ev
    d_weight = total_debt / ev
 
    wacc = e_weight * cost_of_equity + d_weight * cost_of_debt * (1 - tax_rate)
 
    print("\n=== WACC COMPONENTS (from discountingcashflows + Yahoo fallback) ===")
    print(f"Ticker: {ticker}")
    print(f"Equity Value (MarketCap or book estimate, full units): {equity_value:,.0f}" if equity_value is not None else "Equity Value: None")
    print(f"Total Debt (scaled): {total_debt:,.0f}")
    print(f"Cash & Equivalents (scaled): {cash:,.0f}")
    print(f"Net Debt (scaled): {total_debt - cash:,.0f}")
    print(f"Enterprise Value (EV, calculated with scaled data): {ev:,.0f}")
    print(f"Beta (Yahoo fallback): {beta}")
    print(f"Cost of Equity (Rf + beta*RP): {cost_of_equity:.2%}")
    print(f"Interest Expense (from IS, scaled): {interest_expense if interest_expense is not None else 'N/A'}")
    print(f"Cost of Debt: {cost_of_debt:.2%}")
    print(f"Tax Rate used: {tax_rate:.2%}")
    print(f"Weight Equity (E/EV): {e_weight:.2%}")
    print(f"Weight Debt (D/EV): {d_weight:.2%}")
    print(f"--> WACC: {wacc:.2%}\n")
    print(f"Shares Outstanding (Yahoo fallback or calculated, expected full units): {shares_outstanding if shares_outstanding is not None else 'N/A'}")
 
    return {
        "wacc": wacc,
        "equity_value": equity_value,
        "total_debt": total_debt,
        "cash": cash,
        "ev": ev,
        "beta": beta,
        "cost_of_equity": cost_of_equity,
        "cost_of_debt": cost_of_debt,
        "shares_outstanding": shares_outstanding
    }
 
 
# ------------------------------
# Extraction of main financial data (FCF, EPS, Net Income, Equity, Revenue)
# ------------------------------
def get_financials_discounting(ticker):
    bs = get_table_discounting(ticker, "balance-sheet-statement")
    is_df = get_table_discounting(ticker, "income-statement")
    cf = get_table_discounting(ticker, "cash-flow-statement")
 
    data = {}
    fcf_keys = ["Free Cash Flow", "Free cash flow", "FreeCashFlow", "free cash flow"]
    fcf, cf_periods = extract_series_from_row(cf, fcf_keys)
    if fcf is None:
        ocf, ocf_periods = extract_series_from_row(cf, ["Operating Cash Flow", "Operating cashflow", "operating cashflow", "Cash Flow From Operating Activities"])
        capex, capex_periods = extract_series_from_row(cf, ["Capital Expenditure", "Capital Expenditures", "capital expenditure"])
        if ocf is not None and capex is not None:
            n = min(len(ocf), len(capex))
            fcf = [ocf[i] - abs(capex[i]) for i in range(n)]
            cf_periods = ocf_periods[:n]
        else:
             cf_periods = None
 
    data["FCF"] = fcf
    data["FCF_Periods"] = cf_periods
 
    eps, is_periods = extract_series_from_row(is_df, ["EPS", "Diluted EPS", "Basic EPS", "Earnings per Share"])
    data["EPS"] = eps
    data["EPS_Periods"] = is_periods
 
    ni, ni_periods = extract_series_from_row(is_df, ["Net Income", "Net income", "NetLossProfit", "Net loss"])
    data["NetIncome"] = ni
    data["NetIncome_Periods"] = ni_periods
 
    eq, eq_periods = extract_series_from_row(bs, ["Total Equity", "Total shareholders' equity", "Total stockholders' equity", "Total Equity Attributable To Parent"])
    data["Equity"] = eq
    data["Equity_Periods"] = eq_periods
 
    rev, rev_periods = extract_series_from_row(is_df, ["Total Revenue", "Revenue", "Sales"])
    data["Revenue"] = rev
    data["Revenue_Periods"] = rev_periods
 
    return data
 
 
# ------------------------------
# DCF (uses FCF list or predicted series)
# ------------------------------
def dcf_from_fcf(fcf_data, wacc, years=YEARS, terminal_growth=TERMINAL_GROWTH, use_avg=USE_FCF_AVG, avg_years=FCF_AVG_YEARS):
    """
    Calculates DCF from a list of historical FCF or a pandas Series of predicted FCF.
    Assumes FCF values are in full units after scaling.
    """
    if isinstance(fcf_data, pd.Series) and not fcf_data.empty:
        print("\nUsing predicted FCF series for DCF projection.")
        projected_fcf_values = fcf_data.values.tolist()
 
        if len(projected_fcf_values) < years:
            raise ValueError(f"Predicted FCF series only has {len(projected_fcf_values)} values, but {years} years are required for projection.")
 
        pv = 0.0
        for i in range(years):
             pv += projected_fcf_values[i] / ((1 + wacc) ** (i + 1))
 
        last_projected_fcf = projected_fcf_values[-1]
        if wacc <= terminal_growth:
            print(f"Warning: WACC ({wacc:.2%}) is not greater than Terminal Growth ({terminal_growth:.2%}). Terminal Value calculation may be invalid.")
        terminal_value = (last_projected_fcf * (1 + terminal_growth)) / (wacc - terminal_growth) if (wacc - terminal_growth) != 0 else 0
        pv += terminal_value / ((1 + wacc) ** years)
 
        return pv
 
    elif isinstance(fcf_data, list) and fcf_data:
        print("\nUsing historical FCF list with growth assumption for DCF projection.")
        if use_avg and len(fcf_data) >= 1:
            base = np.mean(fcf_data[:min(avg_years, len(fcf_data))])
        else:
            base = fcf_data[0]
        growth = 0.05
 
        pv = 0.0
        fcf_t = base
        for i in range(1, years + 1):
            fcf_t = fcf_t * (1 + growth)
            pv += fcf_t / ((1 + wacc) ** i)
 
        terminal_value = (fcf_t * (1 + terminal_growth)) / (wacc - terminal_growth) if (wacc - terminal_growth) != 0 else 0
        pv += terminal_value / ((1 + wacc) ** years)
        return pv
 
    else:
         raise ValueError("FCF data (list or predicted series) not available for DCF")
 
 
# ------------------------------
# EVA (uses Net Income list or predicted series)
# ------------------------------
def eva_from_financial_data(net_income_data, equity_list, wacc, shares_outstanding, years=YEARS):
    """
    Calculates EVA from Net Income data (list or predicted series) and Equity list.
    Assumes values are in full units after scaling.
    Requires shares_outstanding for calculations based on predicted EPS.
    Returns a dictionary including total_value and per_share_value.
    """
    if not equity_list or len(equity_list) == 0:
        raise ValueError("Book Equity data not available for EVA")
 
    eq_latest = equity_list[0]
 
    if isinstance(net_income_data, pd.Series) and not net_income_data.empty and shares_outstanding is not None and shares_outstanding > 0:
        print("\nUsing predicted Net Income for EVA projection.")
        # The 'net_income_data' passed here is already the projected Net Income
        # calculated outside this function by predicted_income_from_eps.
        # Use the values directly.
        projected_ni_series = net_income_data
        projected_ni_values = projected_ni_series.values.tolist()
 
 
        pv_eva_stream = 0.0
        if len(projected_ni_values) < years:
            raise ValueError(f"Projected Net Income series only has {len(projected_ni_values)} values, but {years} years are required for EVA projection.")
 
 
        for i in range(years):
            projected_ni_t = projected_ni_values[i]
            # Economic Profit (EVA) for year i+1 (since discounting period starts from 1)
            # EP_t = NI_t - (Capital_t-1 * WACC)
            # Using latest Equity value (eq_latest) as Capital_t-1 proxy for all future periods.
            eva_t = projected_ni_t - (eq_latest * wacc)
            discount_factor = (1 + wacc) ** (i + 1)
            pv_eva_stream += eva_t / discount_factor
 
 
        # Total Firm Value = Book Equity + PV(EP stream)
        total_eva_value = eq_latest + pv_eva_stream
 
        per_share_value = total_eva_value / shares_outstanding if shares_outstanding > 0 else None
 
        return {
            "total_value": total_eva_value,
            "calculation_method": "Projected from NI", # Updated method description
            "per_share_value": per_share_value
        }
 
 
    elif isinstance(net_income_data, list) and net_income_data:
        print("\nUsing historical Net Income list for EVA calculation (single period).")
        ni = net_income_data[0]
 
        eva_latest = ni - (eq_latest * wacc)
 
        pv = 0.0
        for i in range(1, years + 1):
             pv += eva_latest / ((1 + wacc) ** i)
 
        per_share_value = pv / shares_outstanding if shares_outstanding is not None and shares_outstanding > 0 else None
 
        return {
            "total_value": pv,
            "calculation_method": "Historical Single Period",
            "per_share_value": per_share_value
        }
 
 
    else:
         raise ValueError("Net Income data (list or predicted series) not available for EVA")
 
def predicted_income_from_eps(predicted_eps_series, shares_outstanding):
    """Calculates predicted Net Income series from predicted EPS series and shares outstanding."""
    if predicted_eps_series is None or predicted_eps_series.empty or shares_outstanding is None or shares_outstanding <= 0:
        return None
 
    predicted_income = predicted_eps_series * shares_outstanding
    return predicted_income
 
# ------------------------------
# Peter Lynch
# ------------------------------
def peter_lynch_from_eps(eps_list, ticker, growth_rate=0.05):
    """Calculates Peter Lynch value using EPS (assumed to be per share already) and a specified growth rate."""
    eps = None
    if eps_list and len(eps_list) > 0:
        eps = eps_list[0]
    else:
        pass # Rely on EPS list
 
    if eps is None or eps <= 0:
        raise ValueError("EPS not available or not positive for Peter Lynch")
 
    # Peter Lynch: P/E = Growth Rate. Intrinsic Value = EPS * Growth Rate (as percentage)
    fair_pe = max(15, growth_rate * 100)
    value = fair_pe * eps
    return value
 
 
# ------------------------------
# EV/Sales using local file benchmark
# ------------------------------
def ev_sales_valuation_from_revenue(ticker, revenue, local_ev_sales_df, total_debt_scaled, cash_scaled, manual_ev_sales=None):
    """Calculates EV/Sales valuation using revenue (assumed to be in full units after scaling). Returns per-share value."""
    stock = yf.Ticker(ticker)
    info = stock.info
    industry_from_yahoo = info.get("industry", "")
 
    try:
        ev_sales_multiple = find_ev_sales_by_industry(industry_from_yahoo, local_ev_sales_df, manual_ev_sales)
    except Exception as e:
        if manual_ev_sales is not None:
            ev_sales_multiple = manual_ev_sales
            print(f"\nError finding EV/Sales multiple for industry '{industry_from_yahoo}' in local file: {e}. Using manual EV/Sales value: {manual_ev_sales}")
        else:
            raise
 
    revenue_most = revenue
    ev = ev_sales_multiple * revenue_most
 
    shares = info.get("sharesOutstanding", None)
    if shares is None:
        mc = info.get("marketCap", None)
        price = info.get("regularMarketPrice", None)
        if mc and price and price > 0:
            shares = int(round(mc / price))
 
    if shares is not None and shares > 0:
        net_debt = total_debt_scaled - cash_scaled
        equity_value_from_ev = ev - net_debt
        return equity_value_from_ev / shares
 
    return ev
 
 
# ------------------------------
# MASTER: executes all and prints
# ===============================
def evaluate_all(ticker):
    valuation_results = {}
 
    wacc_info = calculate_wacc_discounting(ticker, RISK_FREE, MARKET_PREMIUM, TAX_RATE)
    wacc = wacc_info["wacc"]
    shares_out = wacc_info.get("shares_outstanding", None)
    total_debt_scaled = wacc_info.get("total_debt", 0)
    cash_scaled = wacc_info.get("cash", 0)
 
    fin = get_financials_discounting(ticker)
    fcf_list = fin.get("FCF")
    fcf_periods = fin.get("FCF_Periods")
    eps_list = fin.get("EPS")
    eps_periods = fin.get("EPS_Periods")
    ni_list = fin.get("NetIncome")
    eq_list = fin.get("Equity")
    revenue_list = fin.get("Revenue")
 
    if fcf_list:
        fcf_list_scaled = [x * DATA_SCALE_FACTOR for x in fcf_list]
    else:
        fcf_list_scaled = None
    if ni_list:
        ni_list_scaled = [x * DATA_SCALE_FACTOR for x in ni_list]
    else:
        ni_list_scaled = None
    if eq_list:
        eq_list_scaled = [x * DATA_SCALE_FACTOR for x in eq_list]
        print(f"\nNote: Book Equity from discountingcashflows is scaled by {DATA_SCALE_FACTOR:,.0f} to match other financial data units for EVA calculation.")
    else:
        eq_list_scaled = None
 
    if revenue_list:
        revenue_list_scaled = [x * DATA_SCALE_FACTOR for x in revenue_list]
    else:
        revenue_list_scaled = None
 
    print("\n=== Data extracted from discountingcashflows ===")
    print(f"Scale applied (not per item per share): {DATA_SCALE_FACTOR:,.0f} (indicated 'In Millions' on the site)")
    print("Free Cash Flow (recent, scaled):", (fcf_list_scaled[:5] if fcf_list_scaled else "N/A"))
    print("EPS (recent, per share):", (eps_list[:5] if eps_list else "N/A"))
    print("Net Income (recent, scaled):", (ni_list_scaled[:5] if ni_list_scaled else "N/A"))
    print("Book Equity (recent, scaled):", (eq_list_scaled[:5] if eq_list_scaled else "N/A"))
    print("Revenue (recent, scaled):", (revenue_list_scaled[:5] if revenue_list_scaled else "N/A"))
 
    predicted_eps_series = None
    if eps_list and eps_periods:
        predicted_eps_series = predict_regression(eps_list, eps_periods, PROJECTION_YEARS, "EPS", regression_type='logarithmic')
 
    predicted_fcf_series = None
    if fcf_list_scaled and fcf_periods:
        predicted_fcf_series = predict_regression(fcf_list_scaled, fcf_periods, PROJECTION_YEARS, "FCF", regression_type='linear')
 
    predicted_eps_growth_rate = None
    if predicted_eps_series is not None and not predicted_eps_series.empty:
        first_eps = predicted_eps_series.iloc[0]
        last_eps = predicted_eps_series.iloc[-1]
        num_years = len(predicted_eps_series)
        if first_eps is not None and last_eps is not None and first_eps > 0 and num_years > 0:
            if (last_eps / first_eps) >= 0:
                 predicted_eps_growth_rate = (last_eps / first_eps)**(1/num_years) - 1
                 print(f"\nPredicted EPS CAGR over {num_years} years: {predicted_eps_growth_rate:.2%}")
            else:
                 print("\nCannot calculate Predicted EPS CAGR (ratio of last to first predicted EPS is negative).")
        else:
            print("\nCannot calculate Predicted EPS CAGR (first/last predicted EPS is not positive/available or num_years is zero).")
    else:
        print("\nPredicted EPS series not available for CAGR calculation.")
 
    growth_rate_for_plynch = predicted_eps_growth_rate if predicted_eps_growth_rate is not None else 0.05
 
    try:
        fcf_data_for_dcf = predicted_fcf_series if predicted_fcf_series is not None and not predicted_fcf_series.empty else fcf_list_scaled
        total_enterprise_value = dcf_from_fcf(fcf_data_for_dcf, wacc, YEARS, TERMINAL_GROWTH, USE_FCF_AVG, FCF_AVG_YEARS)
        shares_to_use = shares_out
        if shares_to_use is None or shares_to_use <= 0:
            stock = yf.Ticker(ticker)
            info = stock.info
            shares_from_yahoo = info.get("sharesOutstanding", None)
            mc = info.get("marketCap", None)
            price = info.get("regularMarketPrice", None)
            if shares_from_yahoo is not None and shares_from_yahoo > 0:
                 shares_to_use = shares_from_yahoo
            elif mc is not None and price is not None and price > 0:
                      try:
                          shares_to_use = int(round(mc / price))
                          print(f"\nUsing shares outstanding calculated from Market Cap and Price: {shares_to_use}")
                      except:
                          shares_to_use = None
                          print("\nCould not calculate shares outstanding from Market Cap and Price.")
 
        print(f"Note: Shares Outstanding from Yahoo Finance is expected to be in full units ({shares_to_use if shares_to_use is not None else 'N/A'}).")
 
        equity_value_from_dcf_ev = total_enterprise_value - (total_debt_scaled - cash_scaled)
 
        if shares_to_use is not None and shares_to_use > 0:
            fv_per_share = equity_value_from_dcf_ev / shares_to_use
            valuation_results["DCF"] = round(fv_per_share, 1)
        else:
             valuation_results["DCF"] = "N/A (shares not available or zero)"
 
    except Exception as e:
        valuation_results["DCF"] = f"Error: {e}"
 
    global main_eva_result
    main_eva_result = None
    predicted_ni_series = None
 
    try:
        if predicted_eps_series is not None and not predicted_eps_series.empty and shares_out is not None and shares_out > 0:
             predicted_ni_series = predicted_income_from_eps(predicted_eps_series, shares_out)
             main_eva_result = eva_from_financial_data(predicted_ni_series, eq_list_scaled, wacc, shares_out, YEARS)
 
        elif ni_list_scaled and eq_list_scaled:
             if shares_out is None or shares_out <= 0:
                 stock = yf.Ticker(ticker)
                 info = stock.info
                 shares_from_yahoo = info.get("sharesOutstanding", None)
                 mc = info.get("marketCap", None)
                 price = info.get("regularMarketPrice", None)
                 if shares_from_yahoo is not None and shares_from_yahoo > 0:
                      shares_out = shares_from_yahoo
                 elif mc is not None and price is not None and price > 0:
                      try:
                          shares_out = int(round(mc / price))
                          print(f"\nUsing shares outstanding calculated from Market Cap and Price for historical EVA: {shares_out}")
                      except:
                          shares_out = None
                          print("\nCould not calculate shares outstanding from Market Cap and Price for historical EVA.")
 
             if shares_out is not None and shares_out > 0:
                 main_eva_result = eva_from_financial_data(ni_list_scaled, eq_list_scaled, wacc, shares_out, YEARS)
             else:
                 pass
 
    except Exception as e:
        main_eva_result = {"value": f"Error: {e}", "method": "EVA (Error)"}
        print(f"\nError during EVA calculation: {e}")
 
    try:
        growth_rate_for_plynch = predicted_eps_growth_rate if predicted_eps_growth_rate is not None else 0.05
        pl = peter_lynch_from_eps(eps_list, ticker, growth_rate_for_plynch)
        valuation_results["Peter Lynch"] = round(pl, 1)
    except Exception as e:
        valuation_results["Peter Lynch"] = f"Error: {e}"
 
    industry_from_yahoo = ""
    try:
        last_year = datetime.now().year - 1
        local_file_path = LOCAL_EV_SALES_FILE.format(last_year=last_year)
        local_ev_sales_df = load_ev_sales_from_local_file(local_file_path)
 
        if revenue_list_scaled and len(revenue_list_scaled) > 0:
            revenue_most = revenue_list_scaled[0]
            stock = yf.Ticker(ticker)
            info = stock.info
            industry_from_yahoo = info.get("industry", "")
 
            evs_val_per_share = ev_sales_valuation_from_revenue(ticker, revenue_most, local_ev_sales_df, total_debt_scaled, cash_scaled, MANUAL_EV_SALES)
 
            if isinstance(evs_val_per_share, (int, float)):
                 valuation_results["EV/Sales (Local File)"] = round(evs_val_per_share, 1)
            else:
                 valuation_results["EV/Sales (Local File)"] = f"Total EV Value: {round(evs_val_per_share, 1)} (Shares not available)"
 
        else:
            valuation_results["EV/Sales (Local File)"] = "N/A (Revenue not available)"
 
    except FileNotFoundError as e:
         valuation_results["EV/Sales (Local File)"] = f"Error: {e}"
         print(f"\nError loading local EV/Sales file: {e}")
    except Exception as e:
        valuation_results["EV/Sales (Local File)"] = f"Error: {e}"
        print(f"\nError during EV/Sales calculation from local file: {e}")
 
    print(f"\nIndustry identified from Yahoo Finance for EV/Sales valuation (matched against local file data): {industry_from_yahoo if industry_from_yahoo else 'N/A'}")
 
    if isinstance(main_eva_result, dict) and main_eva_result.get("per_share_value") is not None:
         valuation_results[f"EVA ({main_eva_result['calculation_method']})"] = round(main_eva_result["per_share_value"], 1)
    elif isinstance(main_eva_result, dict) and isinstance(main_eva_result.get("value"), str) and "Error" in main_eva_result.get("value"):
         valuation_results[main_eva_result["method"]] = main_eva_result["value"]
    else:
         valuation_results["EVA"] = "N/A (Calculation failed or shares missing)"
 
    print("\n=== Fair Value Estimates per Share ===")
    for method, value in valuation_results.items():
        print(f" - {method}: {value}")
 
 
# ===============================
# EXECUTE
# ===============================
if __name__ == "__main__":
    evaluate_all(TICKER)